In [22]:
!pip install -U FlagEmbedding -q
!pip install gradio --q
!pip install -q llama-index
!pip install -q llama-core
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-huggingface-api
!pip install groq -q
!pip install rank_bm25 -q
!pip install --upgrade huggingface_hub -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
gradio 5.9.0 requires huggingface-hub>=0.25.1, but you have huggingface-hub 0.23.5 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.5 which is incompatible.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

## Imports

In [46]:
import pdfplumber
import re
import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import numpy as np

import nltk
nltk.download('punkt_tab')

from FlagEmbedding import FlagReranker
from groq import Groq
import gradio as gr

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Legion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Loading data, cleaning, chunking

In [2]:
def extract_text_without_header(pdf_path):
    """
    Extract text from a PDF file, removing headers.

    Args:
    ----------
    pdf_path: str
        Path to the PDF file.

    Returns:
    ----------
    text: str
        The extracted text from the PDF.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                # Remove repetitive headers or unwanted page numbers
                page_text = page_text.split("\n", 1)[-1]
            text += page_text + "\n" if page_text else "\n"
    return text


In [3]:
pdf_file_path = "Harry Potter and the Sorcerers Stone.pdf"
pdf_text = extract_text_without_header(pdf_file_path)

In [4]:
def split_to_chapters(text):
    """
    Split the text into chapters using a delimiter ('CHAPTER', for example).

    Args:
    ----------
    text: str
        Full text extracted from the PDF.

    Returns:
    ----------
    chapters: list
        List of chapters as individual strings.
    """
    chapters = re.split(r'\bCHAPTER\s+\d+', text, flags=re.IGNORECASE)
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]  # Clean empty entries
    return chapters


In [5]:
chapters = split_to_chapters(pdf_text)
print(f"Number of chapters: {len(chapters)}")
print(f"Sample chapter content:\n{chapters[0][:500]}")

Number of chapters: 1
Sample chapter content:
CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had



In [6]:
def clean_chapters(chapters):
    """
    Clean chapter text by removing artifacts such as page numbers or line breaks.

    Args:
    ----------
    chapters: list
        List of chapter strings.

    Returns:
    ----------
    cleaned_chapters: list
        List of cleaned chapter strings.
    """
    cleaned_chapters = []
    for chapter in chapters:
        # Remove unwanted line breaks and normalize spacing
        cleaned_chapter = re.sub(r'\n+', ' ', chapter)
        cleaned_chapter = re.sub(r'\s{2,}', ' ', cleaned_chapter)
        cleaned_chapters.append(cleaned_chapter.strip())
    return cleaned_chapters


In [7]:
cleaned_chapters = clean_chapters(chapters)
print(cleaned_chapters[0][:500])


CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had 


In [8]:
def split_to_sections(chapters, max_words=500):
    """
    Split chapters into smaller sections of a defined word limit.

    Args:
    ----------
    chapters: list
        List of chapter strings.
    max_words: int
        Maximum number of words per section.

    Returns:
    ----------
    sections: list
        List of section dictionaries with metadata.
    """
    sections = []
    for chapter_number, chapter in enumerate(chapters, start=1):
        words = chapter.split()
        for i in range(0, len(words), max_words):
            section_text = " ".join(words[i:i + max_words])
            sections.append({
                "chapter_number": chapter_number,
                "section_number": i // max_words + 1,
                "text": section_text
            })
    return sections


In [9]:
sections = split_to_sections(cleaned_chapters)
print(f"Number of sections: {len(sections)}")
print(f"Sample section: {sections[0]}")


Number of sections: 153
Sample section: {'chapter_number': 1, 'section_number': 1, 'text': 'CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you\'d expect to be involved in anything strange or mysterious, because they just didn\'t hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere. The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn\'t think they coul

In [10]:
def save_as_txt(content, filename, directory):
    with open(os.path.join(directory, f"{filename}.txt"), 'w', encoding='utf-8') as file:
        file.write(content)
        
def save_sections_as_files(sections, output_dir, file_format='txt'):
    """
    Save sections into individual files.

    Args:
    ----------
    sections: list
        List of section dictionaries.
    output_dir: str
        Directory where files will be saved.
    file_format: str
        Desired file format (e.g., 'txt').

    Returns:
    ----------
    None
    """
    os.makedirs(output_dir, exist_ok=True)
    for section in sections:
        filename = f"Chapter_{section['chapter_number']}_Section_{section['section_number']}"
        if file_format.lower() == 'txt':
            save_as_txt(section['text'], filename, output_dir)
    print(f"Sections saved in: {output_dir}")


In [11]:
save_sections_as_files(sections, "sections", file_format='txt')

Sections saved in: sections


## Semanthic Search

In [12]:
def create_index_with_metadata(directory_path):
    """
    Create a semantic search index from text files, with enhanced metadata support.

    Args:
    ----------
    directory_path: str
        Path to the directory containing text files.

    Returns:
    ----------
    index: VectorStoreIndex
        The created semantic index.
    metadata_map: dict
        Mapping of document IDs to their metadata (chapter, section).
    """
    embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

    reader = SimpleDirectoryReader(directory_path)
    documents = reader.load_data() 

    # Initialize metadata map
    metadata_map = {}
    for idx, doc in enumerate(documents):
        file_name = doc.metadata.get("file_name", f"Unknown_{idx}")  # Default name if missing
        chapter, section = extract_chapter_section(file_name)

        # Add a unique document ID
        document_id = f"doc_{idx}"
        doc.metadata.update({
            "document_id": document_id,
            "chapter": chapter,
            "section": section,
        })

        metadata_map[document_id] = doc.metadata

    # Create and return the semantic index
    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
    return index, metadata_map

In [13]:
def extract_chapter_section(file_name):
    """
    Extract chapter and section numbers from the file name.

    Args:
    ----------
    file_name: str
        File name (e.g., "Chapter_1_Section_3.txt").

    Returns:
    ----------
    chapter: int
        The chapter number.
    section: int
        The section number within the chapter.
    """
    match = re.match(r"Chapter_(\d+)_Section_(\d+).txt", file_name)
    if match:
        chapter = int(match.group(1))
        section = int(match.group(2))
        return chapter, section
    return None, None

In [14]:
def perform_semantic_search(query, index, metadata_map, top_k=5, score_threshold=0.5):
    """
    Perform semantic search on the index.

    Args:
    ----------
    query: str
        The user query for searching relevant sections.
    index: VectorStoreIndex
        The semantic search index.
    metadata_map: dict
        Mapping of document IDs to metadata.
    top_k: int
        Number of top results to retrieve.
    score_threshold: float
        Minimum similarity score to include a result.

    Returns:
    ----------
    results: list
        List of dictionaries containing matched sections and metadata.
    """
    retriever = index.as_retriever(similarity_top_k=top_k)
    response = retriever.retrieve(query)

    results = []
    for node in response:
        # Fetch metadata for the document
        doc_id = node.node.metadata["document_id"]
        metadata = metadata_map.get(doc_id, {})
        score = node.score

        if score >= score_threshold:
            results.append({
                "text": node.node.text,
                "score": score,
                "chapter": metadata.get("chapter"),
                "section": metadata.get("section"),
            })

    return results

In [15]:
def display_search_results(results):
    """
    Display search results in a user-friendly format.

    Args:
    ----------
    results: list
        List of dictionaries containing search results and metadata.

    Returns:
    ----------
    None
    """
    if not results:
        print("No matching sections found.")
        return

    print(f"Top {len(results)} Results:")
    for idx, result in enumerate(results, start=1):
        print(f"\nResult {idx}:")
        print(f"Chapter: {result['chapter']}, Section: {result['section']}")
        print(f"Score: {result['score']:.2f}")
        print(f"Text: {result['text'][:500]}...")  # Show snippet

In [40]:
directory_path = "sections"
index, metadata_map = create_index_with_metadata(directory_path)

In [53]:
query = "What is Sorcerer's Stone?"
results = perform_semantic_search(query, index, metadata_map, top_k=3, score_threshold=0.3)

# Display results
display_search_results(results)


Top 3 Results:

Result 1:
Chapter: 1, Section: 109
Score: 0.82
Text: Hermione ignored him. "Nicolas Flamel," she whispered dramatically, "is the only known maker of the Sorcerer's Stone!" This didn't have quite the effect she'd expected. "The what?" said Harry and Ron. "Oh, honestly, don't you two read? Look -- read that, there." She pushed the book toward them, and Harry and Ron read: The ancient study of alchemy is concerned with making the Sorcerer's Stone, a 175 any metal into pure gold. It also produces the Elixir of Life, which will make the drinker immorta...

Result 2:
Chapter: 1, Section: 69
Score: 0.79
Text: high into the air as it would go without her leaving her seat, but Harry didn't have the faintest idea what a bezoar was. He tried not to look at Malfoy, Crabbe, and Goyle, who were shaking with laughter. "I don't know, sit." "Thought you wouldn't open a book before coming, 109 cold eyes. He had looked through his books at the Dursleys', but did Snape expect him to rememb

## BM25

In [25]:
def preprocess_corpus(sections):
    """
    Preprocess sections into a tokenized corpus and metadata.

    Args:
    ----------
    sections: list
        List of dictionaries containing text and metadata.

    Returns:
    ----------
    corpus: list
        Tokenized list of words for each section.
    metadata: list
        Metadata for each section.
    """
    corpus = []
    metadata = []
    
    for section in sections:
        text = section["text"].lower()  # Lowercase for consistency
        tokens = word_tokenize(text)   # Tokenize into words
        corpus.append(tokens)
        metadata.append({
            "chapter_number": section["chapter_number"],  
            "section_number": section["section_number"], 
            "text": section["text"]
        })
    
    return corpus, metadata

In [28]:
def bm25_search(query, corpus, metadata, top_n=3):
    """
    Perform BM25 search on the dataset.

    Args:
    ----------
    query: str
        The user query.
    corpus: list
        The tokenized corpus.
    metadata: list
        Metadata corresponding to each section.
    top_n: int
        Number of top results to return.

    Returns:
    ----------
    results: list
        List of top matching documents with scores and metadata.
    """

    bm25 = BM25Okapi(corpus)

    # Tokenize the query
    query_tokens = word_tokenize(query.lower())

    # Compute BM25 scores for all documents
    scores = bm25.get_scores(query_tokens)

    # Get the top N results
    top_indices = np.argsort(scores)[-top_n:][::-1]

    results = []
    for idx in top_indices:
        result = {
            "chapter_number": metadata[idx]["chapter_number"],  # Updated key
            "section_number": metadata[idx]["section_number"],  # Updated key
            "text": metadata[idx]["text"],
            "score": scores[idx]
        }
        results.append(result)

    return results

In [29]:
def display_results_bm25(results):
    """
    Display BM25 search results in a user-friendly format.

    Args:
    ----------
    results: list
        List of dictionaries containing search results.

    Returns:
    ----------
    None
    """
    if not results:
        print("No matching sections found.")
        return

    print(f"Top {len(results)} Results:")
    for idx, result in enumerate(results, start=1):
        print(f"\nResult {idx}:")
        print(f"Chapter: {result['chapter_number']}, Section: {result['section_number']}")
        print(f"Score: {result['score']:.2f}")
        print(f"Text: {result['text'][:500]}...")

In [30]:
corpus, metadata = preprocess_corpus(sections)

query = "What is a Philosopher's Stone?"
results = bm25_search(query, corpus, metadata, top_n=3)

display_results_bm25(results)

Top 3 Results:

Result 1:
Chapter: 1, Section: 109
Score: 11.61
Text: Hermione ignored him. "Nicolas Flamel," she whispered dramatically, "is the only known maker of the Sorcerer's Stone!" This didn't have quite the effect she'd expected. "The what?" said Harry and Ron. "Oh, honestly, don't you two read? Look -- read that, there." She pushed the book toward them, and Harry and Ron read: The ancient study of alchemy is concerned with making the Sorcerer's Stone, a 175 any metal into pure gold. It also produces the Elixir of Life, which will make the drinker immorta...

Result 2:
Chapter: 1, Section: 144
Score: 11.59
Text: mirror? Should I break it?" Harry's mind was racing. What I want more than anything else in the world at the moment, he thought, is to find the Stone before Quirrell does. So if I look in the mirror, I should see myseff finding it -- which means I'll see where it's hidden! But how can I look without Quirrell realizing what I'm up 234 He tried to edge to the left, to ge

## Reranker

In [32]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

def use_reranker(query, documents, top_k=3):
    """
    Use a reranker to refine search results for better relevance.

    Args:
    ----------
    query: str
        The user query for reranking.
    documents: list
        List of dictionaries containing document metadata and text.
    top_k: int
        Number of top results to return after reranking.

    Returns:
    ----------
    top_docs: list
        Reranked list of top matching documents with scores and metadata.
    """

    pairs = [[query, doc["text"]] for doc in documents]
    
    scores = reranker.compute_score(pairs, normalize=True)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    
    # Retrieve the top documents along with their scores
    top_docs = [
        {
            "chapter_number": documents[i].get("chapter_number"),
            "section_number": documents[i].get("section_number"),
            "text": documents[i]["text"],
            "bm25_score": documents[i].get("score"), 
            "reranker_score": scores[i]
        }
        for i in top_indices
    ]
    
    return top_docs

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

C:\Users\Legion\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Legion\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [34]:
def display_reranked_results(results):
    """
    Display reranked search results in a user-friendly format.

    Args:
    ----------
    results: list
        List of dictionaries containing reranked results and metadata.

    Returns:
    ----------
    None
    """
    
    if not results:
        print("No matching sections found.")
        return

    print(f"Top {len(results)} Results (Reranked):")
    for idx, result in enumerate(results, start=1):
        print(f"\nResult {idx}:")
        print(f"Chapter: {result['chapter_number']}, Section: {result['section_number']}")
        print(f"BM25 Score: {result['bm25_score']:.2f}")
        print(f"Reranker Score: {result['reranker_score']:.2f}")
        print(f"Text: {result['text'][:500]}...")

In [33]:
query = "What is the Sorcerer's Stone?"

bm25_results = bm25_search(query, corpus, metadata, top_n=5)

reranked_results = use_reranker(query, bm25_results, top_k=3)


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [35]:
display_reranked_results(reranked_results)

Top 3 Results (Reranked):

Result 1:
Chapter: 1, Section: 109
BM25 Score: 17.48
Reranker Score: 0.79
Text: Hermione ignored him. "Nicolas Flamel," she whispered dramatically, "is the only known maker of the Sorcerer's Stone!" This didn't have quite the effect she'd expected. "The what?" said Harry and Ron. "Oh, honestly, don't you two read? Look -- read that, there." She pushed the book toward them, and Harry and Ron read: The ancient study of alchemy is concerned with making the Sorcerer's Stone, a 175 any metal into pure gold. It also produces the Elixir of Life, which will make the drinker immorta...

Result 2:
Chapter: 1, Section: 112
BM25 Score: 14.63
Reranker Score: 0.34
Text: on? Harry jumped back on his Nimbus Two Thousand and took off. Gliding silently over the castle he saw Snape enter the forest at a run. He followed. The trees were so thick he couldn't see where Snape had gone. He flew in circles, lower and lower, brushing the top branches of trees until he heard voices. He

## LLM

In [36]:
def create_context(docs):
    """
    Create a textual context from the provided documents.

    Args:
    ----------
    docs: list
        List of dictionaries containing document metadata and text.

    Returns:
    ----------
    context: str
        A concatenated string of documents with their metadata.
    """
    context = ''
    for i, doc_info in enumerate(docs, start=1):
        context += f"Document {i} (Chapter {doc_info.get('chapter_number')}, Section {doc_info.get('section_number')}):\n"
        context += doc_info['text'] + "\n\n"
    return context

In [61]:
def perform_rag(query, index, corpus, metadata, method='semantic', top_n=3, rerank=False):
    """
    Perform Retrieval-Augmented Generation by searching for relevant documents.
    """
    if method == 'bm25':
        print("Performing BM25 Search...")
        top_docs = bm25_search(query, corpus, metadata, top_n=top_n)
    elif method == 'semantic':
        print("Performing Semantic Search...")
        top_docs = semantic_search(query, index, top_n=top_n)
    else:
        print("No search method specified.")
        return []

    if rerank:
        print("Performing Reranking...")
        top_docs = use_reranker(query, top_docs, top_k=top_n)

    return top_docs

In [71]:
def perform_llm(query, use_context, groq_key_input, top_docs=None, system_prompt="", temperature=0.7):
    """
    Perform LLM-based question answering with optional context.
    """

    if use_context == 1 and top_docs:
        context = create_context(top_docs)
        llm_query = (
            system_prompt
            + "Question: " + query + "\n"
            + "Context: " + context + "\n"
            + "Answer: "
        )
    else:
        llm_query = (
            system_prompt
            + "Question: " + query + "\n"
            + "Context: []\n"
            + "Answer: "
        )


    client = Groq(api_key=api_key)


    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": llm_query,
        }],
        model="gemma-7b-it",
        temperature=temperature,
    )

    return chat_completion.choices[0].message.content

In [63]:
query = "What is the Sorcerer's Stone?"
system_prompt = "You are a knowledgeable assistant summarizing text from Harry Potter."
top_docs = perform_rag(query, index, corpus, metadata, method='bm25', top_n=5, rerank=True)

Performing BM25 Search...
Performing Reranking...


In [ ]:
# API Key for LLM Service
api_key = "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"

In [47]:
# Generate response using the top documents as context
response = perform_llm(
    query=query,
    use_context=1,
    api_key=api_key,
    top_docs=top_docs,
    system_prompt=system_prompt,
    temperature=0.7
)

print("LLM Response:\n", response)

LLM Response:
 The Sorcerer's Stone is a mythical potion that can turn base metals into gold and grant immortality. It is said to be the most valuable and powerful object in the wizarding world.


## Web Interface

In [72]:
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Harry Potter Book Search and LLM Query System")
        gr.Markdown("This service allows you to perform semantic search, BM25 search, and reranking of documents with an option to use context for querying the LLM. If no search is specified, the LLM will use its own knowledge to answer.")
        
        BOOK_URL = "https://docenti.unimc.it/antonella.pascali/teaching/2018/19055/files/ultima-lezione/harry-potter-and-the-philosophers-stone"  # Replace with actual link to the book
        gr.Markdown(f'<p style="font-size: 18px;">Read the book here: <a href="{BOOK_URL}" style="color: orange; text-decoration: underline;">Harry Potter Book</a></p>')
        
        with gr.Row():
            with gr.Column():
                query_input = gr.Textbox(label="Your Query", placeholder="For example, What is Harry Potter's first spell?")
                
                method_input = gr.Radio(
                    choices=["semantic", "bm25", "no search"], label="Search Method", value="semantic"
                )

                groq_key_input = gr.Textbox(label="Groq API Key", type="password")

                rerank_input = gr.Checkbox(label="Use Reranker", value=False)

                temperature_input = gr.Slider(minimum=0, maximum=1, step=0.1, label="Temperature", value=0.5)
                use_context = gr.Checkbox(label="Use Context for Query", value=True)

            with gr.Column():
                results_output = gr.Textbox(label="Search Results and LLM Answer", interactive=False)

                doc1_output = gr.Textbox(label="Document 1", interactive=False)
                doc2_output = gr.Textbox(label="Document 2", interactive=False)
                doc3_output = gr.Textbox(label="Document 3", interactive=False)

        def choose_prompt(method):
            if method == 'no search':
                return """
                You are a specialized agent answering questions based on your knowledge. Only answer in English.
                """
            else:
                return """
                You are a specialized agent answering questions using only the provided documents. Select the most relevant documents that answer the question, and provide the answer based solely on them. If no documents answer the question, do not provide an answer. If context is given, it will be provided after the question.
                """

        def process_query(query, method, groq_key_input, rerank, temperature, use_context):
            top_docs = perform_rag(query, index, corpus, metadata, method=method, top_n=3, rerank=rerank)
            
            system_prompt = choose_prompt(method)
            answer = perform_llm(query, use_context, groq_key_input=groq_key_input, top_docs=top_docs, system_prompt=system_prompt, temperature=temperature)
            
            docs_texts = [f"Chapter {doc['chapter_number']}, Section {doc['section_number']}. {doc['text']}" for doc in top_docs]
            while len(docs_texts) < 3:
                docs_texts.append("")
            return answer, docs_texts[0], docs_texts[1], docs_texts[2]

        gr.Button("Execute Search and Get Answer").click(
            process_query,
            inputs=[query_input, method_input, groq_key_input, rerank_input, temperature_input, use_context],
            outputs=[results_output, doc1_output, doc2_output, doc3_output]
        )

    return demo

In [73]:
interface = create_interface()
interface.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Performing BM25 Search...
Performing Reranking...
